In [1]:
import napari
%gui qt5

In [2]:
from brainlit.viz import swc
from skimage import io
import sys
import csv
from scipy import ndimage as ndi
import pandas as pd
import numpy as np
import cloudvolume as cv
from cloudvolume.lib import Bbox
sys.path.append("C:/Users/sanik/Anaconda3/envs/brainlight/Lib/site-packages")
from brainlit.utils.ngl_pipeline import NeuroglancerSession

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


In [3]:
s3_path = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
seg_id = 2
mip = 1
df = swc.read_s3(s3_path, seg_id, mip)
df.head()

Downloading: 100%|██████████| 1/1 [00:00<00:00, 19.65it/s]


,sample,structure,x,y,z,r,parent
0,1,0,4713.0,4470.0,3857.0,1.0,-1
1,4,192,4721.0,4445.0,3849.0,1.0,1
2,7,64,4723.0,4446.0,3851.0,1.0,4
3,8,0,4728.0,4449.0,3852.0,1.0,7
4,14,0,4746.0,4445.0,3858.0,1.0,8


In [4]:
G = swc.df_to_graph(df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
print(G.nodes[1]['x'],G.nodes[1]['y'],G.nodes[1]['z'])

Number of nodes: 1650
Number of edges: 1649


Sample 1 coordinates (x,y,z)
4713 4470 3857


In [5]:
paths = swc.graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")

The graph was decomposed into 179 paths


In [6]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', name='Skeleton 2')

<Shapes layer 'Skeleton 2' at 0x149a124d0>

In [7]:
# Create an NGL session to get the bounding box
ngl_sess = NeuroglancerSession(mip = 1)
img, bbbox, vox = ngl_sess.pull_chunk(2, 2, 1, 1, 1)
bbox = bbbox.to_list()
box = (bbox[:3], bbox[3:])
print(bbox)

Downloading:   0%|          | 0/3 [00:00<?, ?it/s]


[6336, 6100, 2184, 6534, 6250, 2340]


In [10]:
G_sub = swc.get_sub_neuron(G, box)
paths_sub = swc.graph_to_paths(G_sub)
viewer = napari.Viewer(ndisplay=3)
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='blue', name='Neighborhood')

ngl = NeuroglancerSession(mip = 1)
seg_id = 2
v_id = 300 
nx = 1
ny = 1
nz = 1
img, bounds, voxel = ngl.pull_chunk(seg_id,v_id,nx,ny,nz)
# overlay corresponding image
#image_path = 'G-002_15312-4400-6448_15840-4800-6656.tif'
#img_comp = io.imread(image_path)
#img_comp = np.swapaxes(img,0,2)
#viewer.add_image(img)

Downloading:   0%|          | 0/3 [00:00<?, ?it/s]


In [13]:
####Neighbourhood stuff
url = "s3://mouse-light-viz/precomputed_volumes/brain1"
ngl_skel = NeuroglancerSession(url+"_segments", mip=1)
segment = ngl_skel.cv.skeleton.get(7)
ver = segment.vertices;
print(ver[3])
dir(ngl_skel)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 20.18it/s]

[8657942.  2503843.5 5514841.5]


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_voxel',
 'chunk_size',
 'cv',
 'mip',
 'pull_bounds_img',
 'pull_bounds_seg',
 'pull_chunk',
 'pull_voxel',
 'push',
 'scales',
 'url']

In [17]:
voxel = ngl_skel._get_voxel(7, 2)
offset = np.array([1,1,1])
bounds = Bbox(np.subtract(voxel,offset),np.add(voxel,offset))
box = bounds.to_list()
box_inp = (box[:3], box[3:])
img = ngl_skel.pull_bounds_img(bounds)
print(box_inp)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


([14507, 4102, 2791], [14509, 4104, 2793])


In [16]:
G_sub = swc.get_sub_neuron(G, box_inp)
paths_sub = swc.graph_to_paths(G_sub)
print(paths_sub)

[]


In [15]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='blue', name='neighborhood')

IndexError: index 2 is out of bounds for axis 0 with size 2